# Getting Started

## Running Beamlime

Run the `beamlime` command, and specify which workflow to use.
Usually this would be done from a terminal.
The current setup plots the results of the workflow and saves them to a file, which can we specify using the `--image-path` flag:

In [ ]:
!beamlime \
    --workflow dummy \
    --nexus-template-path ../../tests/applications/ymir_detectors.json \
    --nexus-file-path 'ymir_detectors.nxs' \
    --image-path-prefix reduction-result

The dummy workflow returns a single result named "reduction-result.png", which is added as a suffix to the output file name:

![Plot of reduction result](reduction-result.png)

## Adding workflows

Python packages can add workflows as plugins to Beamlime.
To do this, create a Python package with a `beamlime.stateless` entry point in the `pyproject.toml` file.
This is how the "dummy" workflow is added to Beamlime:

```python
# In beamlime's pyproject.toml
[project.entry-points."beamlime.stateless"]
dummy = "beamlime.stateless_workflow:dummy_workflow"
```

Above, 'dummy' is the name of the workflow (which can then be passed to `beamlime --workflow`), and 'beamlime.stateless_workflow' is the name of the Python package and module containing the workflow.
As an example, for a Loki workflow plugin provided by `esssans`, the entry point might look as follows:

```python
# In esssans' pyproject.toml
[project.entry-points."beamlime.stateless"]
ess-loki = "ess.sans.loki:live_workflow"
```

where `live_workflow` must adhere to the [beamlime.StatelessWorkflow](https://scipp.github.io/beamlime/generated/classes/beamlime.StatelessWorkflow.html) protocol:

In [ ]:
import inspect

from beamlime import LiveWorkflow

print(inspect.getsource(LiveWorkflow))

This can be fulfilled by a class or a function, as long as it takes a `JSONGroup` and returns a [beamlime.WorkflowResult](https://scipp.github.io/beamlime/generated/classes/beamlime.WorkflowResult.html).